In [1]:
import os
import pandas as pd
import json
os.chdir("..")
%pwd

'/home/jupyter/crisp'

In [2]:
from main import run
from synthetic.facebook_synthetic_data_generator import generator_example
from synthetic.facebook_synthetic_data import DATASETS

In [9]:
# synthetic dataset parameters 
n_example = 1
dim_inv=5
n_bin=1
dim_spu=10
n_exp_train=3000 #int(2e2)
n_exp_test=300
n_env=3
save_dir= 'data/synthetic'
test=False 

In [10]:
output_data_regime = ["real-valued", "binary", "binary", "real-valued", "multi-class"][n_example-1]
verbose = 1

In [11]:
# (re) create synthetic dataset for experiment
generator_example(n_example, dim_inv, dim_spu, n_exp_train, n_env ,save_dir, test=False)#, n_bin) # generate pickle with train data
generator_example(n_example, dim_inv, dim_spu, n_exp_test, n_env ,save_dir, test=True)  # generate pickle with test data
outfile_train = os.path.join(save_dir,"data_fb_example_%s_dim_inv_%s_dim_spu_%s_n_exp_%s_n_env_%s_test_%s.pickle" %(n_example, dim_inv, dim_spu, n_exp_train, n_env,"False"))
outfile_test = os.path.join(save_dir,"data_fb_example_%s_dim_inv_%s_dim_spu_%s_n_exp_%s_n_env_%s_test_%s.pickle" %(n_example, dim_inv, dim_spu, n_exp_test, n_env,"True"))

Environments variables: {'E0': 0.1, 'E1': 1.5, 'E2': 2}
Generated Synthetic Data according to the Facebook setup Example: 1
Environments variables: {'E0': 0.1, 'E1': 1.5, 'E2': 2}
Generated Synthetic Data according to the Facebook setup Example: 1


In [12]:
df_train = pd.read_pickle(outfile_train)
df_test = pd.read_pickle(outfile_test)

In [13]:
df_train.head()

,Causal_0,Causal_1,Causal_2,Causal_3,Causal_4,Non_causal_0,Non_causal_1,Non_causal_2,Non_causal_3,Non_causal_4,Non_causal_5,Non_causal_6,Non_causal_7,Non_causal_8,Non_causal_9,env_split,Subj_ID,Target
0,0.043423,-0.080249,-0.129519,-0.075018,-0.131197,0.104379,0.049981,-1.496275,1.109402,-0.190706,0.472563,1.546026,0.589012,-0.326063,-0.261328,0.0,0.0,-0.187895
1,-0.021883,-0.243507,-0.007291,0.023292,0.079689,0.744490,0.513087,-0.916191,-1.602201,0.026719,0.142905,-0.293738,-0.242381,0.547993,-0.664380,0.0,1.0,0.133622
2,-0.018484,-0.037015,-0.121028,-0.062270,-0.046372,1.221464,-1.695731,1.305152,0.902147,-1.043561,-0.296319,-0.533380,1.991536,-0.572924,-0.545621,0.0,2.0,-0.279760
3,0.192178,-0.127877,0.009295,-0.066610,0.060805,-0.332814,2.871652,0.028257,1.024367,0.125901,-1.249411,-2.622004,-0.880377,0.141732,-0.639234,0.0,3.0,-0.180793
4,-0.073002,-0.088338,-0.041891,-0.080483,-0.101631,0.604842,1.663523,-0.653143,-0.599665,0.784399,-0.994145,-0.531333,-0.208100,-1.322123,-0.527459,0.0,4.0,0.560201


In [15]:
# define config
config = {
    "name": "Example Experiment for AH casual ensemble",
    "short_name": "ah_experiment_notebook",
    "bucket_project": "fdl-us-astronaut-health",
    "bucket_name": "ah_21_data",
    "bucket_path": "gs://ah_21_data",
    "verbose": verbose,
    "test_val_split": [0.1, 0.1],
    "per_variant_experiment": False,
    "data_options": {
        #'dataset_fp': '../data/test_multiclass.pkl',
        'synthetic_train_test_split' : True,
        #'dataset_fp' : outfile,
        'dataset_fp_train' : outfile_train,
        'dataset_fp_test' : outfile_test,
        'output_data_regime' : output_data_regime,
        'subject_keys': 'Subj_ID',
        'targets': ['Target'],
        #'predictors': ['All'],
        'predictors': list(df_train.columns[0:dim_inv+dim_spu]),
        'environments': ['env_split'],
        'exclude': ['Subj_ID']
    },
    "feature_selection_options": {
        "max_features": 20,
        "verbose": verbose,
        "seed": 12
    },
    "ensemble_options": {
        "models": ["ICP", "DCF", "LIRM", "NLICP"],
        "LIRM" : {
            "use_icp_initialization": False,
            "verbose": verbose,
            "n_iterations": 1000,
            "seed": 0,
            "lr": 0.001,
            "cuda": False,
            "output_data_regime" : output_data_regime,
        },
        "DCF" : {
            "minP": 0.1,
            "maxP": 0.9,
            "minFeatures": 1,
            "minAccuracy": 0.5,
            "seed": 0,
            "verbose": verbose,
            "target": ["Target"],
            "output_pvals": False,
            "output_data_regime" : output_data_regime,
        },
        "NLICP" : {
            "max_set_size": 2,
            "alpha": 0.05,
            "seed": 12,
            "verbose": verbose,
            "method": "MLP",
            "hidden_dim": 256,
            "output_data_regime" : output_data_regime,
        },
        "LICP" : {
            "max_set_size": 2,
            "alpha": 0.05,
            "seed": 12,
            "verbose": verbose,
            "output_data_regime" : output_data_regime,
        },
    },
    "use_cloud":False,
    "results_directory": "results/"
}

In [16]:
config.get("ensemble_options").get("LIRM")

{'use_icp_initialization': False,
 'verbose': 1,
 'n_iterations': 1000,
 'seed': 0,
 'lr': 0.001,
 'cuda': False,
 'output_data_regime': 'real-valued'}

In [17]:
# write config
with open('experiment_configs/ensemble_test_run.json', 'w') as f:
    json.dump(config, f)

In [ ]:
# run crisp
run(config)

Running a per sample experiment
Loaded  3  train environments
Env  0  has  802  samples
X shape  (802, 15)  y shape  (802, 1)
Env  1  has  795  samples
X shape  (795, 15)  y shape  (795, 1)
Env  2  has  803  samples
X shape  (803, 15)  y shape  (803, 1)
Loaded val set, X shape: (600, 15)  y shape:  (600, 1)
Loaded test set, X shape: (300, 15)  y shape:  (300, 1)
Running Ensemble with the following models:  ['ICP', 'DCF', 'LIRM', 'NLICP']
Checking for features with zero variance
Across all environments the following columns had zero variance: []
Running a per sample experiment


/home/jupyter/crisp/utils/CorrelationToTarget.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  all_x = np.array(all_x)
/home/jupyter/crisp/utils/CorrelationToTarget.py:25: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  all_y = np.array(all_y)


Loaded  3  train environments
Env  0  has  802  samples
X shape  (802, 15)  y shape  (802, 1)
Env  1  has  795  samples
X shape  (795, 15)  y shape  (795, 1)
Env  2  has  803  samples
X shape  (803, 15)  y shape  (803, 1)
Loaded val set, X shape: (600, 15)  y shape:  (600, 1)
Loaded test set, X shape: (300, 15)  y shape:  (300, 1)
Running correlation analysis
Intersection: []
Running a per sample experiment
Loaded  3  train environments
Env  0  has  802  samples
X shape  (802, 15)  y shape  (802, 1)
Env  1  has  795  samples
X shape  (795, 15)  y shape  (795, 1)
Env  2  has  803  samples
X shape  (803, 15)  y shape  (803, 1)
Loaded val set, X shape: (600, 15)  y shape:  (600, 1)
Loaded test set, X shape: (300, 15)  y shape:  (300, 1)
Running IRM (Feature Reduction Mode)
NLIRM using MLP
0               0.70720         0.00666         0.00083      
100             -0.30807        0.01165         0.17468      
200             0.05094         0.01165         0.00017      
300             0

/home/jupyter/crisp/models/TorchMultiClassDeconfounder.py:206: UserWarning: tfp.edward2 module is deprecated and will be removed on 2019-12-01. Use https://github.com/google/edward2 library instead.
  log_joint = ed.make_log_joint_fn(self.ppca_model)




Inferred W:
[[-0.04124932  0.04384892  0.12382874  0.48749894 -0.68781024  0.55963427
  -0.62026256  0.63698536  0.6342491   0.11792096  0.690178    0.36124846
  -0.06458571 -0.25930217  0.7431929 ]]
Standard Deviation:
[[0.00645824 0.00702116 0.00828922 0.00928005 0.0096931  0.00759559
  0.00656211 0.00795376 0.00810371 0.0082995  0.00760848 0.00805026
  0.00755903 0.01141375 0.00613793]]
pval: 0.028794416243654825
Inferred W:
[[-0.0176532  -0.09239312  0.19741516  0.653254   -0.5504196   0.60771763
  -0.6408285   0.55009776  0.63292575  0.18333547  0.7084795   0.27612793
  -0.10897168 -0.278386    0.7460957 ]]
Standard Deviation:
[[0.00909354 0.00827078 0.00866392 0.00893712 0.00875882 0.00904487
  0.00804825 0.00858639 0.00868768 0.00786387 0.0098697  0.00731125
  0.01096533 0.0085676  0.00740273]]


In [ ]:
config['data_options']['dataset_fp']